In [1]:
import sys
sys.path.append('../src')
from config import *

In [2]:
import melt
melt.init_flags()
FLAGS = melt.get_flags()
FLAGS.pretrained = '../input/tf-xlm-roberta-base'
melt.init()

2020-04-18 15:21:27 0:00:00 fcntl.floc with lock_file /root/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2020-04-18 15:21:27 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-04-18 15:21:27 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-04-18 15:21:27 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 gpu: [5] CUDA_VISIABLE_DEVICES=[] work_mode: train distributed: False horovod: False
2020-04-18 15:21:28 0:00:01 model: [melt] model_dir: [/tmp/melt]


In [3]:
from model import xlm_model as Model

I0418 15:21:29.170642 140643527042880 file_utils.py:41] PyTorch version 1.4.0 available.
I0418 15:21:29.171661 140643527042880 file_utils.py:57] TensorFlow version 2.2.0-dev20200417 available.


In [4]:
model = Model()

2020-04-18 15:21:29 0:00:01 load xlm_model from ../input/tf-xlm-roberta-base start
I0418 15:21:29.338798 140643527042880 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0418 15:21:29.340036 140643527042880 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_emb

In [5]:
model.weights[-1]

<tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [6]:
model.load_weights('../working/exps/v1/debug/model_weight.h5')
model.weights[-1]

<tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.0012], dtype=float32)>

In [7]:
import gezi
from dataset import Dataset
ds = Dataset('valid').make_batch(32, gezi.list_files('../input/tfrecords/xlm/validation'))
next(iter(ds))

({'id': <tf.Tensor: shape=(32,), dtype=string, numpy=
  array([b'1001', b'786', b'4136', b'6119', b'3069', b'7360', b'674',
         b'3357', b'7971', b'1675', b'5234', b'7231', b'7546', b'6990',
         b'6385', b'7390', b'5065', b'7426', b'6031', b'543', b'6841',
         b'6361', b'7382', b'239', b'3213', b'4826', b'5139', b'1141',
         b'5720', b'134', b'5759', b'2803'], dtype=object)>,
  'identity_hate': <tf.Tensor: shape=(32, 1), dtype=float32, numpy=
  array([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]], dtype=float32)>,
  'input_word_ids': <tf.Tensor: sha

In [8]:
from evaluate import evaluate
from husky.callbacks import EvalCallback
eval_callback = EvalCallback(model, ds, evaluate)

In [9]:
eval_callback.eval()

2020-04-18 15:22:45 0:01:18 eval_step: 1 step: 0 epoch: 0.0
2020-04-18 15:22:45 0:01:18 valid_metrics:
 +----------+-----------+
|     loss |   auc/all |
|----------+-----------|
| 0.450762 |   0.53091 |
+----------+-----------+


In [10]:
import tensorflow as tf
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/debug'))
model.weights[-1]

<tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.0012], dtype=float32)>

In [11]:
eval_callback.y

array([0., 0., 0., ..., 1., 0., 0.], dtype=float32)

In [12]:
eval_callback.x

{'id': array(['1001', '786', '4136', ..., '2', '2732', '5023'], dtype='<U4')}